####1-Arabinda-900-BLR-2-Hari-800-BBSR-3-Shyam-100-BLS convert to columns 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Create DataFrame and Insert Data") \
    .getOrCreate()




In [0]:
dbutils.fs.put('/Filestore/tables/regular_expression_intreview.txt','1-Arabinda-900-BLR-2-Hari-800-BBSR-3-Shyam-100-BLS',overwrite=True)
df=spark.read.format("text")\
    .option("header","True")\
        .load('/Filestore/tables/regular_expression_intreview.txt')
#pattern = r"(\d*)-(\w*)-(\d*)-(\w*)"

#df=df.drop(col("value"))



Wrote 50 bytes.


####Approach-1

In [0]:
df1=df.withColumn("New_Value",regexp_replace("value",r"(.*?\-){4}","$0,"))
df1=df.withColumn("New_Value",regexp_replace("value",r"([^\-]*\-{4})","$0,"))


df2=df1.withColumn("New_Vlaue1",explode(split(df1["New_Value"],'-,')))
df3=df2.withColumn("ID",split(df2["New_Vlaue1"],'-')[0])\
    .withColumn("Name",split(df2["New_Vlaue1"],'-')[1])\
        .withColumn("Sales",split(df2["New_Vlaue1"],'-')[2])\
            .withColumn("Location",split(df2["New_Vlaue1"],'-')[3])
df_final=df3.drop("value","New_Value","New_Vlaue1")
df_final.show(truncate=0)

+---+--------+-----+--------+
|ID |Name    |Sales|Location|
+---+--------+-----+--------+
|1  |Arabinda|900  |BLR     |
+---+--------+-----+--------+



####Approach-2

 ####regular expression pattern:

[^\\-]*: Matches any character except a hyphen ([^\\-]), zero or more times (*).

\\-: Matches a hyphen -.

Together, ([^\\-]*\\-){4} matches any characters followed by a hyphen exactly four times.

In [0]:
pattern = r"([^\-]*\-){4}"
df11=df.withColumn("New_Value",regexp_replace("value", pattern, "$0,"))



df22=df11.withColumn("New_Vlaue1",explode(split(df11["New_Value"],'-,')))
df33=df22.withColumn("ID",split(df22["New_Vlaue1"],'-')[0])\
    .withColumn("Name",split(df22["New_Vlaue1"],'-')[1])\
        .withColumn("Sales",split(df22["New_Vlaue1"],'-')[2])\
            .withColumn("Location",split(df22["New_Vlaue1"],'-')[3])
df_final_2=df33.drop("value","New_Value","New_Vlaue1")
df_final_2.show()

+---+--------+-----+--------+
| ID|    Name|Sales|Location|
+---+--------+-----+--------+
|  1|Arabinda|  900|     BLR|
|  2|    Hari|  800|    BBSR|
|  3|   Shyam|  100|     BLS|
+---+--------+-----+--------+

